# GOAL

- Evaluar el rendimiento del pipeline RAG **baseline**
- Evaluar el rendimiento del pipeline RAG **con reranker**

Utilizaremos la librería [`ragas`](https://docs.ragas.io/en/latest/getstarted/) para evaluar el rendimiento del pipeline RAG.
y la librería [`Reranker`](https://github.com/AnswerDotAI/rerankers) para reordenar los documentos relevantes.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv

load_dotenv()

file_path = (
    "data/biblioteca-de-alimentos.pdf"
)
loader = PyPDFLoader(file_path)
docs = loader.load()

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

In [5]:
from langchain_ollama import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
import os
#embeddings = OllamaEmbeddings(model="llama3")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
from langchain_qdrant import QdrantVectorStore
from langchain_qdrant import RetrievalMode

qdrant = QdrantVectorStore.from_documents(
    splits,
    embedding=embeddings,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
)

In [7]:
retriever = qdrant.as_retriever()

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import OllamaLLM
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-3.5-turbo") 

# Define prompt template
template = """Utilize the retrieved context below to answer the question.
If you're unsure of the answer, simply state you don't know and apologies
Keep your response concise, limited to two sentences.
Question: {question}
Context: {context}
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [10]:
rag_chain.invoke("What is the purpose of the regulation?")

'The purpose of the regulation is to review and revise the regulations regarding the regularization of food exempt from registration.'

## Creación del dataset de preguntas y respuestas

In [18]:
QA_generation_prompt = ChatPromptTemplate.from_template("""
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}
Output:::""")

# Crear una cadena para crear una pregunta
question_chain = (
    {"context": RunnablePassthrough()}
    | QA_generation_prompt
    | llm
    | StrOutputParser()
)

### Obtener una muestra aleatoria de documentos

In [19]:
import random
from tqdm import tqdm

sampled_docs = random.sample(docs, 15)
sampled_docs_processed = [doc.page_content for doc in sampled_docs]

#### Generar preguntas y respuestas

In [20]:

questions = [question_chain.invoke({"context": sampled_context}) for sampled_context in tqdm(sampled_docs_processed)]

















100%|██████████| 15/15 [00:13<00:00,  1.11it/s]


In [21]:
questions

['Factoid question: What is the purpose of RDC 604/2022?\nAnswer: Enriquecimento obrigatório do sal com iodo e das farinhas de trigo e de milho com ferro e ácido fólico destinados ao consumo humano.',
 'Factoid question: What is the theme of Regulatory Theme 3.5 in the 2024/2025 Regulatory Agenda?\nAnswer: Reavaliação da autorização de uso do aditivo alimentar dióxido de titânio em alimentos.',
 'Factoid question: What does Lei 10.674/2003 require products to inform consumers about?\nAnswer: The presence of gluten.',
 'Factoid question: What is the document related to the procedures for Requests for Inclusion and Extension of Use of Food Additives and Manufacturing Technology Co-Adjuvants in Brazilian Legislation?\nAnswer: Guia nº 43, versão 1, de 14/12/2020 - Procedimentos para Pedidos de Inclusão e Extensão de Uso de Aditivos Alimentares e Coadjuvantes de Tecnologia de Fabricação na Legislação Brasileira.',
 'Factoid question: What is the theme of Regulatory Agenda 3.31 in the contex

In [22]:
questions_processed = []
ground_truth = []
for question in questions:
    questions_processed.append(question.split("Factoid question: ")[-1].split("Answer: ")[0])
    ground_truth.append(question.split("Factoid question: ")[-1].split("Answer: ")[1])

In [23]:
questions_processed

['What is the purpose of RDC 604/2022?\n',
 'What is the theme of Regulatory Theme 3.5 in the 2024/2025 Regulatory Agenda?\n',
 'What does Lei 10.674/2003 require products to inform consumers about?\n',
 'What is the document related to the procedures for Requests for Inclusion and Extension of Use of Food Additives and Manufacturing Technology Co-Adjuvants in Brazilian Legislation?\n',
 'What is the theme of Regulatory Agenda 3.31 in the context provided?\n',
 'What is the document related to "Critérios gerais e classificação de materiais para embalagens em contato com alimentos"?\n\n',
 'What are the procedures for regularização de alimentos?\n',
 'What is the regulation number for the prohibition of importing, manufacturing, distributing, and commercializing foods and beverages based on Aloe Vera in Brazil?\n\n',
 'What are the requirements for enriching wheat and corn flour with iron and folic acid?\n',
 'What is the theme of Regulatory Agenda 2024/2025 related to the regularizatio

In [24]:
ground_truth

['Enriquecimento obrigatório do sal com iodo e das farinhas de trigo e de milho com ferro e ácido fólico destinados ao consumo humano.',
 'Reavaliação da autorização de uso do aditivo alimentar dióxido de titânio em alimentos.',
 'The presence of gluten.',
 'Guia nº 43, versão 1, de 14/12/2020 - Procedimentos para Pedidos de Inclusão e Extensão de Uso de Aditivos Alimentares e Coadjuvantes de Tecnologia de Fabricação na Legislação Brasileira.',
 'Atualização periódica da lista de padrões microbiológicos para alimentos.',
 'RDC 91/2001',
 'Procedimentos para regularização de alimentos',
 'RE 5052/2011',
 'RDC 714/2022 - Requisitos sanitários para enriquecimento e restauração de alimentos.',
 'Theme Regulatório 3.22 da Agenda Regulatória 2024/2025: Revisão da regulamentação sobre regularização de alimentos dispensados de registro.',
 'Atualizada em 06.11.2024',
 'RE 1/2002',
 '23',
 'In the sector of foods.',
 'IN 75/2020']

In [25]:
# Parsear preguntas y respuestas
contexts = []
answers = []
# Inferencia
for query in questions_processed:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])


/var/folders/2y/w7q6dzmn0xvd8k73mz2tz8f40000gn/T/ipykernel_10494/2952356377.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])


In [26]:
data = {
    "question": questions,
    "answer": answers,
    "reference": ground_truth,
    "retrieved_contexts": contexts
}

In [27]:
from datasets import Dataset

# Convert dict to dataset
dataset = Dataset.from_dict(data)

  0%|          | 0/15 [01:18<?, ?it/s]


In [28]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [29]:
result = evaluate(
    dataset = dataset,
    llm=llm,
    embeddings=embeddings,
    metrics=[
        context_recall,
        faithfulness,
        answer_relevancy,
        context_precision,
    ],)

df = result.to_pandas()
df

Evaluating: 100%|██████████| 60/60 [00:29<00:00,  2.05it/s]


,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,answer_relevancy,context_precision
0,Factoid question: What is the purpose of RDC 6...,[Alterada por: \nRDC 612/2022 \nDocumento rela...,I don't have enough information to determine t...,Enriquecimento obrigatório do sal com iodo e d...,1.0,0.333333,0.000000,0.000000
1,Factoid question: What is the theme of Regulat...,[Equivalente (AREE) \n \nTema Regulatório 3.1 ...,I don't have enough information to determine t...,Reavaliação da autorização de uso do aditivo a...,1.0,1.000000,0.000000,0.500000
2,Factoid question: What does Lei 10.674/2003 re...,[de 2003. \n \nLei 10.674/2003 – Obriga a que...,Lei 10.674/2003 requires food products to info...,The presence of gluten.,1.0,1.000000,0.805121,1.000000
3,Factoid question: What is the document related...,[Aditivos Alimentares e Coadjuvantes de Tecnol...,The document related to the procedures for Req...,"Guia nº 43, versão 1, de 14/12/2020 - Procedim...",1.0,NaN,0.858942,1.000000
4,Factoid question: What is the theme of Regulat...,[Equivalente (AREE) \n \nTema Regulatório 3.1 ...,The theme of Regulatory Agenda 3.31 in the pro...,Atualização periódica da lista de padrões micr...,0.0,0.000000,0.000000,0.000000
5,Factoid question: What is the document related...,[RDC 91/2001 – Critérios gerais e classificaçã...,"The document ""Critérios gerais e classificação...",RDC 91/2001,1.0,0.500000,0.797576,1.000000
6,Factoid question: What are the procedures for ...,[1.1. Procedimentos para regularização de alim...,The procedures for regularização de alimentos ...,Procedimentos para regularização de alimentos,1.0,0.666667,0.738733,0.000000
7,Factoid question: What is the regulation numbe...,"[benzopireno, com identificação do lote e ou d...",The regulation number for the prohibition of i...,RE 5052/2011,1.0,1.000000,0.885096,0.833333
8,Factoid question: What are the requirements fo...,[Perguntas e Respostas sobre Enriquecimento de...,The requirements for enriching wheat and corn ...,RDC 714/2022 - Requisitos sanitários para enri...,1.0,1.000000,0.645704,1.000000
9,Factoid question: What is the theme of Regulat...,[Tema Regulatório 3.22 da Agenda Regulatória 2...,The theme of the Regulatory Agenda 2024/2025 r...,Theme Regulatório 3.22 da Agenda Regulatória 2...,1.0,1.000000,0.664575,1.000000


In [30]:
df.to_csv("baseline_ragas_results.csv", index=False)

In [31]:
# get mean of the metrics column by column
print("Mean Faithfulness: ", round(df["faithfulness"].mean(), 4))
print("Mean Answer relevancy: ", round(df["answer_relevancy"].mean(), 4))
print("Mean Context recall: ", round(df["context_recall"].mean(), 4))
print("Mean Context precision: ", round(df["context_precision"].mean(), 4))


Mean Faithfulness:  0.7857
Mean Answer relevancy:  0.6221
Mean Context recall:  0.9333
Mean Context precision:  0.7444


# EVALUACION DEL RAG AVANZADO

In [32]:

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from datasets import Dataset
from main_app_v3_hybrid import main_query  # RAG avanzado
from basic_rag import basic_rag_pipeline  # RAG básico
from main_app_v2 import main_basic  # RAG básico
# from intermediate_rag import intermediate_rag_pipeline  # RAG intermedio (desactivado)

/Users/carloszurita/Documents/czurita_fork/Castro_Cofre_Zurita/src/embedding/embedding.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_name)


Base de datos vectorial 'demo_collection' creada y 348 documentos añadidos.


/Users/carloszurita/Documents/czurita_fork/Castro_Cofre_Zurita/src/augmented/llm.py:22: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/Users/carloszurita/Documents/czurita_fork/Castro_Cofre_Zurita/src/augmented/llm.py:41: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


In [33]:

# Inicialización de resultados para los tres pipelines
contexts_basic = []
answers_basic = []

contexts_advanced = []
answers_advanced = []

file_path = "data/biblioteca-de-alimentos.pdf"

for query in questions:
    # RAG Básico
    #response_basic, context_basic = basic_rag_pipeline(query, file_path=file_path)
        # RAG Avanzado
    response_advanced, ranked_docs_advanced = main_query(query)
    answers_advanced.append(response_advanced if response_advanced else "Não sei.")
    contexts_advanced.append(
        [doc.page_content for doc in ranked_docs_advanced] if ranked_docs_advanced else ["Nenhum contexto recuperado."]
    )

Fetching 29 files: 100%|██████████| 29/29 [00:00<00:00, 116508.44it/s]


Results passed to Reranker: ['Sanitários para enriquecimento e restauração de alimentos RDC 604/2022 Enriquecimento obrigatório do sal com iodo e das farinhas de trigo e de milho com ferro e ácido fólico destinados ao consumo com ferro e ácido fólico destinados ao consumo humano. Alterada por: RDC 612/2022 Documento relacionado: Perguntas e Respostas sobre Enriquecimento de Farinhas de Trigo e de Milho com Ferro e Ácido de Farinhas de Trigo e de Milho com Ferro e Ácido Fólico. RDC 714/2022 Requisitos sanitários para enriquecimento e restauração de alimentos. Alterada por: RDC 839/2023 Comprovação de segurança e', 'farinhas integrais, massas alimentícias e pães RDC 604/2022 Enriquecimento obrigatório do sal com iodo e das farinhas de trigo e de milho com ferro e ácido fólico destinados ao consumo humano. e ácido fólico destinados ao consumo humano. Alterada por: RDC 612/2022 Documento relacionado: Perguntas e Respostas sobre Enriquecimento de Farinhas de Trigo e de Milho com Ferro e Áci

Fetching 29 files: 100%|██████████| 29/29 [00:00<00:00, 132355.62it/s]


Results passed to Reranker: ['alimentares e coadjuvantes de tecnologia Tema Regulatório 3.5 da Agenda Regulatória 2024/2025: Reavaliação da autorização de uso do aditivo alimentar dióxido de titânio em alimentos. Tema Regulatório dióxido de titânio em alimentos. Tema Regulatório 3.9 da Agenda Regulatória 2024/2025: Regulamentação dos aditivos corantes formulados e da rotulagem de corantes e aromatizantes em alimentos de corantes e aromatizantes em alimentos embalados. Tema Regulatório 3.14 da Agenda Regulatória 2024/2025: Revisão da regulamentação de aditivos alimentares e coadjuvantes de tecnologia autorizados', 'Regulatório 3.28 da Agenda Regulatória 2024/2025: Atualização periódica da lista das partes de vegetais autorizadas para o preparo de chás e para uso como especiarias. RDC 716/2022 Requisitos', 'de Alimentos Tema Regulatório 3.3 da Agenda Regulatória 2024/2025: Identificação de estratégias para promover o acesso a informações necessárias ao consumo seguro de alimentos por pes

Fetching 29 files: 100%|██████████| 29/29 [00:00<00:00, 122246.05it/s]


...

Results passed to Reranker: ['Sanitária RDC 172/2017 Procedimentos para a importação e a exportação de bens e produtos destinados à pesquisa científica ou tecnológica e à pesquisa envolvendo seres humanos. 5.1. Procedimentos para envolvendo seres humanos. 5.1. Procedimentos para importação de alimentos RDC 281/2003 – Exigir como procedimento de importação para “aceite de orujo de oliva” ou óleo de bagaço e ou caroço de oliva,', '............... 23 5. Controle sanitário em comércio exterior e ambientes de portos, aeroportos, fronteiras e recintos alfandegados 23 5.1. Procedimentos para importação de alimentos', 'e identificação como integral e para destaque da presença de ingredientes integrais. Documento relacionado: Perguntas e Respostas sobre Cereais Integrais RDC 839/2023 Comprovação de segurança e', 'para uso em alimentos. Guia nº 23, versão 1, de 23/07/2019 Guia para Comprovação da Segurança de Alimentos e Ingredientes. Documento relacionado: Perguntas e Respostas sobre Solicitaçõ

Fetching 29 files: 100%|██████████| 29/29 [00:00<00:00, 156948.15it/s]


Results passed to Reranker: ['BIBLIOTECA DE ALIMENTOS Atualizada em 06.11.2024 Coordenação de Processos Regulatórios – CPROR Assessoria de Melhoria da Qualidade Regulatória – ASREG Gabinete do Diretor-Presidente AGENDA REGULATÓRIA', 'à análise pelos setores técnicos da Anvisa Alterada por: RDC 23/2015 RDC 208/2018 RDC 25/2011 – Procedimentos gerais para utilização dos serviços de protocolo de documentos no âmbito da Anvisa de protocolo de documentos no âmbito da Anvisa Alterada por: RDC 50/2013 Atos relacionados: RDC 470/2021 Procedimentos para o recebimento de documentos em suporte eletrônico. IN 50/2019 Lista de', 'de ingestão controlada de açúcares. Alterada por: RDC 818/2023 Requisitos sanitários dos adoçantes de mesa e dos adoçantes dietéticos. RDC 839/2023 Comprovação de segurança e autorização de uso de', 'sanitária ................................ ................................ ...22 4. Controle, fiscalização e monitoramento de produtos e serviços ............................

Fetching 29 files: 100%|██████████| 29/29 [00:00<00:00, 150352.06it/s]


Results passed to Reranker: ['401/2020 Altera a RDC 22/2015 para atualizar as referências de especificações para compostos fontes de nutrientes e outras substâncias para uso em fórmulas para nutrição enteral. RDC 429/2020 em fórmulas para nutrição enteral. RDC 429/2020 Rotulagem nutricional dos alimentos embalados. Altera a RDC 21/2015. RDC 839/2023 Comprovação de segurança e autorização de uso de novos alimentos e e autorização de uso de novos alimentos e novos ingredientes. Altera a RDC 22/2015. Documento relacionado: Perguntas e Respostas sobre Fórmulas para Nutrição Enteral. RDC 460/2020 Requisitos para Nutrição Enteral. RDC 460/2020 Requisitos sanitários das fórmulas dietoterápicas para erros inatos do metabolismo. Alterada por: RDC 429/2020 Rotulagem nutricional dos alimentos embalados. RDC', 'da rotulagem nutricional nos alimentos embalados. Documentos relacionados: Perguntas e Respostas sobre Rotulagem Nutricional de Alimentos embalados Perguntas e Respostas sobre Alegações Nut

In [34]:
data_advanced = {
    "question": questions,
    "answer": answers_advanced,
    "reference": ground_truth,
    "retrieved_contexts": contexts_advanced,
}


dataset_advanced = Dataset.from_dict(data_advanced)

In [35]:
result_advanced = evaluate(
    dataset=dataset_advanced,
    llm=None,
    embeddings=None,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
)

Evaluating: 100%|██████████| 60/60 [00:31<00:00,  1.90it/s]


In [36]:
df_advanced = result_advanced.to_pandas()

df_advanced.to_csv("advanced_ragas_results.csv", index=False)
df_advanced.head()

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision
0,Factoid question: What is the purpose of RDC 6...,"[farinhas integrais, massas alimentícias e pãe...",Enriquecimiento obligatorio de la sal con yodo...,Enriquecimento obrigatório do sal com iodo e d...,1.000000,0.843394,1.0,1.00
1,Factoid question: What is the theme of Regulat...,[alimentares e coadjuvantes de tecnologia Tema...,Reevaluación de la autorización de uso del adi...,Reavaliação da autorização de uso do aditivo a...,1.000000,0.852076,1.0,1.00
2,Factoid question: What does Lei 10.674/2003 re...,[Portaria MJ 2.658/2003 – Define o símbolo de ...,La presencia de gluten.,The presence of gluten.,1.000000,0.879255,1.0,1.00
3,Factoid question: What is the document related...,"[para uso em alimentos. Guia nº 23, versão 1, ...","Guía n.º 43, versión 1, del 14/12/2020 - Proce...","Guia nº 43, versão 1, de 14/12/2020 - Procedim...",0.833333,0.818737,1.0,0.45
4,Factoid question: What is the theme of Regulat...,[Perguntas e Respostas sobre Solicitações Pós ...,Actualización periódica de la lista de estánda...,Atualização periódica da lista de padrões micr...,1.000000,0.867015,1.0,1.00


In [37]:
print("\nRAG Avançado:")
print("Mean Faithfulness: ", round(df_advanced["faithfulness"].mean(), 4))
print("Mean Answer Relevancy: ", round(df_advanced["answer_relevancy"].mean(), 4))
print("Mean Context Recall: ", round(df_advanced["context_recall"].mean(), 4))
print("Mean Context Precision: ", round(df_advanced["context_precision"].mean(), 4))


RAG Avançado:
Mean Faithfulness:  0.8333
Mean Answer Relevancy:  0.7186
Mean Context Recall:  0.9333
Mean Context Precision:  0.9267


## Adding a reranker step

In [ ]:
query = "What is the purpose of the regulation?"

retrieved_docs = retriever.get_relevant_documents(query, kwargs={"k": 10})


In [34]:
qdrant.similarity_search_with_score(query, k=10)

[(Document(metadata={'source': '../practicos-rag/data/benchmark_data/Reglamento 1333 2008.pdf', 'page': 3, '_id': 'da42de520e5e4c6ba84f2f9795589abd', '_collection_name': 'my_documents'}, page_content='Regulation and to adopt appropriate transitional measures. Since those measures are\nof general scope and are designed to amend non-essential elements of this Regulation,'),
  0.46665629111211765),
 (Document(metadata={'source': '../practicos-rag/data/benchmark_data/Reglamento 1333 2008.pdf', 'page': 5, '_id': '67b487e0039d4aaa8e56eda9d6dbb2a5', '_collection_name': 'my_documents'}, page_content='HAVE ADOPTED THIS REGULATION:\nCHAPTER I\nSUBJECT MATTER, SCOPE AND DEFINITIONS\nArticle 1\nSubject matter\nThis Regulation lays down rules on food additives used in foods with a view to ensuring'),
  0.4592167729175051),
 (Document(metadata={'source': '../practicos-rag/data/benchmark_data/Reglamento 1333 2008.pdf', 'page': 6, '_id': 'b01e219628454a078aa6a971839b0aae', '_collection_name': 'my_docu

In [25]:
retrieved_docs

[Document(metadata={'source': '../practicos-rag/data/benchmark_data/Reglamento 1333 2008.pdf', 'page': 3, '_id': 'da42de520e5e4c6ba84f2f9795589abd', '_collection_name': 'my_documents'}, page_content='Regulation and to adopt appropriate transitional measures. Since those measures are\nof general scope and are designed to amend non-essential elements of this Regulation,'),
 Document(metadata={'source': '../practicos-rag/data/benchmark_data/Reglamento 1333 2008.pdf', 'page': 5, '_id': '67b487e0039d4aaa8e56eda9d6dbb2a5', '_collection_name': 'my_documents'}, page_content='HAVE ADOPTED THIS REGULATION:\nCHAPTER I\nSUBJECT MATTER, SCOPE AND DEFINITIONS\nArticle 1\nSubject matter\nThis Regulation lays down rules on food additives used in foods with a view to ensuring'),
 Document(metadata={'source': '../practicos-rag/data/benchmark_data/Reglamento 1333 2008.pdf', 'page': 6, '_id': 'b01e219628454a078aa6a971839b0aae', '_collection_name': 'my_documents'}, page_content='Regulation (EC) No 1333/200

In [26]:
import cohere as co
cohere_client = co.Client(os.getenv("COHERE_API_KEY"))
def rerank_docs(query, retrieved_docs):
    reranked_docs = cohere_client.rerank(
        model="rerank-english-v3.0",
        query=query,
        documents=retrieved_docs,
        rank_fields=["page_content"],
        return_documents=True
    )
    return reranked_docs

from rerankers import Reranker

def open_source_reranker(query, retrieved_docs):
    #reranker = Reranker('cross-encoder', verbose=0,model_type='cross-encoder')
    reranker = Reranker("colbert", verbose=0)
    retrieved_docs = [doc.page_content for doc in retrieved_docs]
    reranked_docs = reranker.rank(query, retrieved_docs)
    return reranked_docs


In [27]:
reranked_docs = open_source_reranker(query, retrieved_docs)

Loading default colbert model for language en
Default Model: colbert-ir/colbertv2.0
Loading ColBERTRanker model colbert-ir/colbertv2.0 (this message can be suppressed by setting verbose=0)
No device set
Using device mps
No dtype set
Using dtype torch.float32
Loading model colbert-ir/colbertv2.0, this might take a while...
Linear Dim set to: 128 for downcasting


In [28]:
reranked_docs.results

[Result(document=Document(document_type='text', text='Regulation and to adopt appropriate transitional measures. Since those measures are\nof general scope and are designed to amend non-essential elements of this Regulation,', base64=None, image_path=None, doc_id=0, metadata={}), score=0.9069003462791443, rank=1),
 Result(document=Document(document_type='text', text='HAVE ADOPTED THIS REGULATION:\nCHAPTER I\nSUBJECT MATTER, SCOPE AND DEFINITIONS\nArticle 1\nSubject matter\nThis Regulation lays down rules on food additives used in foods with a view to ensuring', base64=None, image_path=None, doc_id=1, metadata={}), score=0.7887976169586182, rank=2),
 Result(document=Document(document_type='text', text='Regulation (EC) No 1333/2008 of the European Parliament and of the Council of...\nDocument Generated: 2023-12-07\n87\nChanges to legislation: \nThere are outstanding changes not yet made to Regulation (EC) No 1333/2008 of the European', base64=None, image_path=None, doc_id=3, metadata={})

In [29]:
contexts = []
answers = []
# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    retrieved_docs = retriever.get_relevant_documents(query)
    reranked_docs = open_source_reranker(query, retrieved_docs)
    if reranked_docs.results:  # Check if there are any results
        contexts.append([reranked_docs.results[0].document.text])

data = {
    "question": questions,
    "answer": answers,
    "reference": ground_truth,
    "retrieved_contexts": contexts
}

Loading default colbert model for language en
Default Model: colbert-ir/colbertv2.0
Loading ColBERTRanker model colbert-ir/colbertv2.0 (this message can be suppressed by setting verbose=0)
No device set
Using device mps
No dtype set
Using dtype torch.float32
Loading model colbert-ir/colbertv2.0, this might take a while...
Linear Dim set to: 128 for downcasting
Loading default colbert model for language en
Default Model: colbert-ir/colbertv2.0
Loading ColBERTRanker model colbert-ir/colbertv2.0 (this message can be suppressed by setting verbose=0)
No device set
Using device mps
No dtype set
Using dtype torch.float32
Loading model colbert-ir/colbertv2.0, this might take a while...
Linear Dim set to: 128 for downcasting
Loading default colbert model for language en
Default Model: colbert-ir/colbertv2.0
Loading ColBERTRanker model colbert-ir/colbertv2.0 (this message can be suppressed by setting verbose=0)
No device set
Using device mps
No dtype set
Using dtype torch.float32
Loading model c

In [30]:
reranked_dataset = Dataset.from_dict(data)
result = evaluate(
    dataset = reranked_dataset,
    llm=llm,
    embeddings=embeddings,
    metrics=[
        context_recall,
        faithfulness,
        answer_relevancy,
        context_precision,
    ],)
reranked_df = result.to_pandas()
reranked_df.to_csv("reranked_ragas_results.csv", index=False)


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [31]:
# get mean of the metrics column by column
print("Mean Faithfulness: ", round(reranked_df["faithfulness"].mean(), 4))
print("Mean Answer relevancy: ", round(reranked_df["answer_relevancy"].mean(), 4))
print("Mean Context recall: ", round(reranked_df["context_recall"].mean(), 4))
print("Mean Context precision: ", round(reranked_df["context_precision"].mean(), 4))

Mean Faithfulness:  0.66
Mean Answer relevancy:  0.6593
Mean Context recall:  0.8667
Mean Context precision:  0.8667
